# Reject images in a database given the global RMS stats


In [35]:
from tkp.db.model import Image
from sqlalchemy import create_engine, not_
from sqlalchemy.orm import sessionmaker
from scipy.stats import norm
from tkp.db.quality import reject, reject_reasons

### settings

In [36]:
host = 'localhost'
user = 'gijs'
password = 'gijs'
database = 'gijs'
dataset_id = 1
print_query = False
est_sigma = 4

### setup connections and get images

In [40]:
param = ('postgresql', user, password, host, 5432, database)
alchemy_engine = create_engine('%s://%s:%s@%s:%s/%s' % param, echo=print_query)
Session = sessionmaker(bind=alchemy_engine)
session = Session()

In [37]:
def reject_bad_rms(session, dataset_id, est_sigma):
    images = session.query(Image.rms_qc).filter(Image.dataset_id == dataset_id).all()
    mu, sigma = norm.fit(images)
    t_low = mu - sigma * est_sigma
    t_high = mu + sigma * est_sigma
    bad_images = session.query(Image).filter(not_(Image.rms_qc.between(t_low, t_high))).all()
    reason = reject_reasons['rms']
    for b in bad_images:
        comment = "RMS of {} is outside range of ({:.3f}, {:.3f}), est_sigma={}".format(b.rms_qc, t_low, t_high, est_sigma)
        reject(b.id, reason, comment, session)
        
reject_bad_rms(session, dataset_id, est_sigma)
session.commit()